In [1]:
### Tratamiento de datos
import glob
import os
import pandas as pd
import numpy as np
import calendar
from contextlib import contextmanager
from datetime import datetime
import math

In [2]:
os.chdir('D:/Reporting/Bases Manuales/09 FORECAST/Oficiales/Sell out')
A = []
csv_files = glob.glob('*.xlsx')

for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='FCST')#,dtype={'Sugerido D+2':np.float64,'Sugerido revisado D + 2':np.float64})
    data['F_DR']=filename
    A.append(data)
A = pd.concat(A,ignore_index=True)
A['Id'] = A.Canal.astype(str) +"-"+ A.CD.astype(str) +"-"+ A.SKU.astype(str) +"-"+ A.F_DR.astype(str)

################### Armado del DataSet ####################
A = A.drop(['Bottler'], axis=1)
A = A.drop(['Region'], axis=1)
A = A.drop(['Marca'], axis=1)
A = A.drop(['Calibre'], axis=1)
A = A.drop(['Pack'], axis=1)
A = A.drop(['Descripcion'], axis=1)
A = A.drop(['SKU 7'], axis=1)
A = A.drop(['SKU 6'], axis=1)
A = A.drop(['SKU 5'], axis=1)
A = A.drop(['SKU 4'], axis=1)
A = A.drop(['SKU 3'], axis=1)
A = A.drop(['SKU 2'], axis=1)
A = A.drop(['SKU'], axis=1)
A = A.drop(['Canal'], axis=1)
A = A.drop(['Canal BGT'], axis=1)
A = A.drop(['Cadena'], axis=1)
A = A.drop(['CD'], axis=1)
A = A.drop(['F_DR'], axis=1)
A = A.drop(['Región'], axis=1)
A = A.drop(['Descripción'], axis=1)
A = A.drop(['Troncal'], axis=1)
A = A.drop(['Unidades'], axis=1)
A = A.drop(['DESCRIPCION'], axis=1)
A = A.drop(['TIPO'], axis=1)
A = A.drop(['Origen'], axis=1)
A = A.drop(['Tipo'], axis=1)

A = A.set_index(('Id'))
A = A.stack()
A = pd.DataFrame(A).reset_index()
t = A.Id.str.split("-", n=4, expand=True)

A["Canal"] = t[0]
A["CD"] = t[1]
A["SKU"] = t[2]
A["Fecha_DR"] = t[3]
t = A.level_1.str.split(" ", n=1, expand=True)
A["Fecha_FCST"] = t[0]
A["Prediccion"] = t[1]
t = A.SKU.str.split(".", n=1, expand=True)
A["SKU"] = t[0]
t = A.Fecha_DR.str.split(".", n=1, expand=True)
A["Fecha_DR"] = t[0]
A.rename(columns = {0:'FCST'} ,inplace=True)
A = A[["Fecha_DR", "Fecha_FCST","Prediccion", "CD", "Canal", "SKU", "FCST"]]

################## LIMPIEZA ###############
### FCST [HL]
A.FCST = A.FCST.astype(float)
A = A[A["FCST"]>0]
A.FCST = A.FCST.round(3)
A.FCST = A.FCST*1000
A.FCST = A.FCST.round(0)
### CD
A["CD"] = A.CD.str.replace(r"ú","u")
A["CD"] = A.CD.str.replace(r"ó","o")
A["CD"] = A.CD.str.replace(r"ñ","n")
A["CD"] = A.CD.str.replace(r"Depósito" ,"")
A["CD"] = A.CD.str.replace(r"Planta ","")
A["CD"] = A.CD.str.replace(r"Villarica","Villarrica")
A["CD"] = A.CD.str.replace(r" ","")
A["CD"] = A.CD.str.lower()
A = A.drop(A[A['CD']=='nan'].index)
A = A.drop(A[A['CD']=='depositocoyhaique'].index)
A = A.drop(A[A['CD']=='laserena'].index)
A = A.drop(A[A['CD']=='santiago'].index)
A["CD"] = A.CD.str.replace(r"villarica","villarrica")
A["CD"] = A.CD.str.replace(r"viñadelmar","vinadelmar")
#### CANAL
A["Canal"] = A.Canal.str.lower()
A["Canal"] = A.Canal.str.replace(r" ","")
A["Canal"] = A.Canal.str.replace(r"-","")
A["Canal"] = A.Canal.str.replace(r"supermercadosregionales","supermercados")
A["Canal"] = A.Canal.str.replace(r"supermercadosr","supermercados")

A = A.groupby(by = ["Fecha_DR", "Fecha_FCST","Prediccion", "CD", "Canal", "SKU"])["FCST"].sum().reset_index()
A = pd.DataFrame(A)

A = A.dropna(subset=['SKU'])
A.to_csv("D:/Reporting/ETL/11 FCST/FCST_DR.txt")

M1 = A[A["Prediccion"]=="M1"]
M1 = M1[["Fecha_FCST","CD", "Canal", "SKU", "FCST"]]
M1['Id'] = M1.Fecha_FCST.astype(str) +"-"+ M1.CD.astype(str) +"-"+ M1.Canal.astype(str) +"-"+ M1.SKU.astype(str)
M1 = M1.drop(M1[M1['SKU']=='nan'].index)

############# SO Mensual ############
os.chdir('D:/Reporting/ETL/01 SELL/Sell Out')
SO = pd.read_csv('Mensual.txt',encoding='ANSI', sep=',')# ,decimal=',')#,dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')

os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Nueva mastra SKU_PEPO.xlsx', sheet_name='Hoja1', dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','SKU_TRUCK': 'SKU','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU = Maestro_SKU[["SKU","Simil"]]
Maestro_SKU = Maestro_SKU.drop_duplicates(['SKU'])

SO = pd.merge(SO,Maestro_SKU ,on=["SKU"],how="outer")
SO = SO.dropna(subset=['A_Mes'])
SO = SO.drop(SO[SO['SKU']=='nan'].index)

SO.Hl = SO.Hl.astype(float)
SO = SO[SO["Hl"]>0]

Maestro_SKU.SKU = Maestro_SKU.SKU.astype(str)
Maestro_SKU.Simil = Maestro_SKU.Simil.astype(str)
t = Maestro_SKU.Simil.str.split(".", n=1, expand=True)
Maestro_SKU["Simil"] = t[0]

M1 = A[A["Prediccion"]=="M1"]
M1 = M1[["Fecha_FCST","CD", "Canal", "SKU", "FCST"]]
M1 = pd.merge(M1,Maestro_SKU ,on=["SKU"],how="outer")
M1 = M1.dropna(subset=['Fecha_FCST'])
M1['Id'] = M1.Fecha_FCST.astype(str) +"-"+ M1.CD.astype(str) +"-"+ M1.Canal.astype(str) +"-"+ M1.Simil.astype(str)

M1.FCST = M1.FCST.astype(float)
M1 = M1[M1["FCST"]>0]

SO.A_Mes = SO.A_Mes.astype(str)
t = SO.A_Mes.str.split(".", n=1, expand=True)
SO["A_Mes"] = t[0]
SO['Id'] = SO.A_Mes.astype(str) +"-"+ SO.CD.astype(str) +"-"+ SO.Seg.astype(str) +"-"+ SO.Simil.astype(str)

################ ACC ######################
IDM1 = M1[["Id"]]
IDSO = SO[["Id"]]
ID = IDM1.append(IDSO)
ID = ID.drop_duplicates(['Id'])

M1r = M1[["Id","FCST"]]
SOr = SO[["Id","Hl"]]

X1 = pd.merge(ID,SOr ,on=["Id"],how="outer")
Acc = pd.merge(X1,M1r ,on=["Id"],how="outer")

t = Acc.Id.str.split("-", n=3, expand=True)
Acc["Fecha_AM"] = t[0]
Acc["CD"] = t[1]
Acc["Canal"] = t[2]
Acc["SKU"] = t[3]
Acc = Acc[["Id", "Fecha_AM", "CD", "Canal", "SKU", "FCST", "Hl"]]
Acc = Acc.dropna(subset=['SKU'])
Acc = Acc.drop(Acc[Acc['SKU']=='nan'].index)
Acc = Acc.fillna(0)
Acc.to_csv("D:/Reporting/ETL/11 FCST/Acc_men.txt")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'D:/Reporting/Bases Manuales/09 FORECAST/Oficiales/Sell out'

In [ ]:
Acc

In [ ]:
A